In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_col= application_df["APPLICATION_TYPE"].value_counts()
app_col

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Determine which values to replace if counts are less than ...?
replace_application= app_col[app_col < 500].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
cal_col=application_df["CLASSIFICATION"].value_counts()
cal_col.head(50)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1256        2
C3200        2
C1246        2
C1267        2
C1234        2
C3700        1
C1732        1
C4120        1
C4500        1
C2570        1
Name: CLASSIFICATION, dtype: int64

In [15]:
# Replace Bins into dataframe- CLASSIFICATION
replace_class= cal_col[cal_col<500].index.tolist()

for clase in replace_class:
    application_df.CLASSIFICATION=application_df.CLASSIFICATION.replace(clase, "Other")
    
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [16]:
# Generate our categorical variable lists
application_ctg= application_df.dtypes[application_df.dtypes =="object"].index.tolist()
application_df[application_ctg].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
dtype: int64

In [18]:
# Create a OneHotEncoder instance 
enc= OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder unsing categorical list 
encode_df= pd.DataFrame(enc.fit_transform(application_df[application_ctg]))

# Add the encoded variable names to the dataframe 
encode_df.columns= enc.get_feature_names(application_ctg)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# Merge One-Hot encoded features and drop the originals
prep_application_df= application_df.merge(encode_df,left_index=True, right_index=True).drop(columns= application_ctg)
prep_application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Attemp 1: Drop Noisy Features, 2 Hidden Layers(80, 30 neurons)

In [21]:
# Going to delete columns that are unnecessary and run NN
print(len(prep_application_df.columns))

44


In [22]:
prep_application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M

In [23]:
# Going to Drop the following:
temp_application_df= prep_application_df.drop(columns= ["USE_CASE_Other","AFFILIATION_Other"])
temp_application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [24]:
# Split the preprocessed data 
y=temp_application_df["IS_SUCCESSFUL"].values
X=temp_application_df.drop(columns=["IS_SUCCESSFUL"]).values

X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=.2, random_state= 3)

In [25]:
# Create StandArdScaler 
scaler= StandardScaler()

# Fit the StandardScaler
X_scaler= scaler.fit(X_train)

# Scale the data 
X_train_scaled= X_scaler.transform(X_train)
X_test_scaled= X_scaler.transform(X_test)

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train[0])
hidden_nodes1 = 80
hidden_nodes2 = 30
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim = num_input,
                                    activation ="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes1, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 80)                3360      
_________________________________________________________________
dense_1 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 81        
Total params: 9,921
Trainable params: 9,921
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

# Create callback that saves every 5 epochs
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a Callback that saves the weights every 5 epochs
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)

# Train the model
fit_model= nn.fit(X_train_scaled, y_train, epochs=200, verbose=1, callbacks=[cp_callback])

#Evaluate results
model_loss, model_accuracy= nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss:{model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
858/858 [==============================] - 4s 3ms/step - loss: 0.5704 - accuracy: 0.7236
Epoch 2/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5579 - accuracy: 0.7287
Epoch 3/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5539 - accuracy: 0.7310
Epoch 4/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5523 - accuracy: 0.7325
Epoch 5/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5514 - accuracy: 0.7307
Epoch 6/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5503 - accuracy: 0.7308
Epoch 7/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5489 - accuracy: 0.7315
Epoch 8/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5478 - accuracy: 0.7342
Epoch 9/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5477 - accuracy: 0.7334
Epoch 10/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5472 - accura

858/858 [==============================] - 3s 3ms/step - loss: 0.5370 - accuracy: 0.7403
Epoch 82/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5364 - accuracy: 0.7402
Epoch 83/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5362 - accuracy: 0.7396
Epoch 84/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5367 - accuracy: 0.7390
Epoch 85/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5363 - accuracy: 0.7392
Epoch 86/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5364 - accuracy: 0.7396
Epoch 87/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5364 - accuracy: 0.7396
Epoch 88/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5365 - accuracy: 0.7391
Epoch 89/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5367 - accuracy: 0.7403
Epoch 90/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5364 - accuracy: 

858/858 [==============================] - 4s 4ms/step - loss: 0.5339 - accuracy: 0.7395
Epoch 161/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5336 - accuracy: 0.7398
Epoch 162/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5343 - accuracy: 0.7401
Epoch 163/200
858/858 [==============================] - 4s 5ms/step - loss: 0.5342 - accuracy: 0.7398
Epoch 164/200
858/858 [==============================] - 4s 5ms/step - loss: 0.5340 - accuracy: 0.7400
Epoch 165/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5347 - accuracy: 0.7399
Epoch 166/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5334 - accuracy: 0.7414
Epoch 167/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5335 - accuracy: 0.7402
Epoch 168/200
858/858 [==============================] - 5s 6ms/step - loss: 0.5335 - accuracy: 0.7400
Epoch 169/200
858/858 [==============================] - 3s 3ms/step - loss: 0.5337 - a

## Attempt 2: 3 Hidden Layers(80, 30,15 neurons)

In [28]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train[0])
hidden_nodes1 = 80
hidden_nodes2 = 30
hidden_node3= 15

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim = num_input,
                                    activation ="relu"))
# Second hidden layer
nn2.add(tf.keras.layers.Dense(units = hidden_nodes1, activation="relu"))

# Third Hidden Layer
nn2.add(tf.keras.layers.Dense(units= hidden_node3, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                3360      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_5 (Dense)              (None, 15)                1215      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 16        
Total params: 11,071
Trainable params: 11,071
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model 
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

# Create a Callback that saves the weights every 5 epochs
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)

# Train the model
fit_model= nn2.fit(X_train_scaled, y_train, epochs=200, verbose=1, callbacks=[cp_callback])

# Evaluate Results
model_loss, model_accuracy= nn2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss:{model_loss}, Accuracy: {model_accuracy}")

Epoch 1/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5690 - accuracy: 0.7239
Epoch 2/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5556 - accuracy: 0.7297
Epoch 3/200
858/858 [==============================] - 4s 5ms/step - loss: 0.5532 - accuracy: 0.7321
Epoch 4/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5509 - accuracy: 0.7315
Epoch 5/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5499 - accuracy: 0.7334
Epoch 6/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5495 - accuracy: 0.7316
Epoch 7/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5482 - accuracy: 0.7335
Epoch 8/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5477 - accuracy: 0.7329
Epoch 9/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7344
Epoch 10/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5464 - accura

858/858 [==============================] - 4s 5ms/step - loss: 0.5354 - accuracy: 0.7401
Epoch 81/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5356 - accuracy: 0.7404
Epoch 82/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5353 - accuracy: 0.7403
Epoch 83/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5352 - accuracy: 0.7394
Epoch 84/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5354 - accuracy: 0.7395
Epoch 85/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5351 - accuracy: 0.7396
Epoch 86/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5352 - accuracy: 0.7406
Epoch 87/200
858/858 [==============================] - 5s 6ms/step - loss: 0.5350 - accuracy: 0.7395
Epoch 88/200
858/858 [==============================] - 5s 5ms/step - loss: 0.5350 - accuracy: 0.7398
Epoch 89/200
858/858 [==============================] - 4s 5ms/step - loss: 0.5352 - accuracy: 

858/858 [==============================] - 3s 4ms/step - loss: 0.5325 - accuracy: 0.7412
Epoch 160/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5333 - accuracy: 0.7410
Epoch 161/200
858/858 [==============================] - 4s 5ms/step - loss: 0.5325 - accuracy: 0.7408
Epoch 162/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5326 - accuracy: 0.7412
Epoch 163/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5330 - accuracy: 0.7409
Epoch 164/200
858/858 [==============================] - 4s 5ms/step - loss: 0.5326 - accuracy: 0.7419
Epoch 165/200
858/858 [==============================] - 4s 5ms/step - loss: 0.5326 - accuracy: 0.7407
Epoch 166/200
858/858 [==============================] - 4s 4ms/step - loss: 0.5345 - accuracy: 0.7412
Epoch 167/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5321 - accuracy: 0.7408
Epoch 168/200
858/858 [==============================] - 3s 4ms/step - loss: 0.5323 - a

## Attempt 3: Redistribute neurons

In [32]:
# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input = len(X_train[0])
hidden_nodes1 = 80
hidden_nodes2 = 35
hidden_node3= 10

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes1, input_dim = num_input,
                                    activation ="relu"))
# Second hidden layer
nn3.add(tf.keras.layers.Dense(units = hidden_nodes1, activation="relu"))

# Third Hidden Layer
nn3.add(tf.keras.layers.Dense(units= hidden_node3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 80)                3360      
_________________________________________________________________
dense_12 (Dense)             (None, 80)                6480      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                810       
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 10,661
Trainable params: 10,661
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model 
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

# Create a Callback that saves the weights every 5 epochs
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"
cp_callback= ModelCheckpoint(filepath=checkpoint_path,save_weights_only=True, save_freq=5)

# Train the model
fit_model= nn3.fit(X_train_scaled, y_train, epochs=400, verbose=1, callbacks=[cp_callback])

# Evaluate Results
model_loss, model_accuracy= nn3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss:{model_loss}, Accuracy: {model_accuracy}")

Epoch 1/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5699 - accuracy: 0.7223
Epoch 2/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5564 - accuracy: 0.7299
Epoch 3/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5532 - accuracy: 0.7310
Epoch 4/400
858/858 [==============================] - 5s 6ms/step - loss: 0.5514 - accuracy: 0.7321
Epoch 5/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5505 - accuracy: 0.7318
Epoch 6/400
858/858 [==============================] - 6s 7ms/step - loss: 0.5487 - accuracy: 0.7328: 0s - loss: 0.5476 - accuracy - ETA: 0s
Epoch 7/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5482 - accuracy: 0.7324
Epoch 8/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5476 - accuracy: 0.7339
Epoch 9/400
858/858 [==============================] - 5s 6ms/step - loss: 0.5471 - accuracy: 0.7334
Epoch 10/400
858/858 [=============================

858/858 [==============================] - 3s 4ms/step - loss: 0.5360 - accuracy: 0.7396
Epoch 81/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5361 - accuracy: 0.7397
Epoch 82/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5358 - accuracy: 0.7400
Epoch 83/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5356 - accuracy: 0.7398
Epoch 84/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5358 - accuracy: 0.7397
Epoch 85/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5355 - accuracy: 0.7395
Epoch 86/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5355 - accuracy: 0.7397
Epoch 87/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5357 - accuracy: 0.7393
Epoch 88/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5352 - accuracy: 0.7398
Epoch 89/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5354 - accuracy: 

858/858 [==============================] - 4s 4ms/step - loss: 0.5335 - accuracy: 0.7403
Epoch 160/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5340 - accuracy: 0.7406
Epoch 161/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5340 - accuracy: 0.7406
Epoch 162/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5337 - accuracy: 0.7404
Epoch 163/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5328 - accuracy: 0.7396
Epoch 164/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5340 - accuracy: 0.7407
Epoch 165/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5327 - accuracy: 0.7406
Epoch 166/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5335 - accuracy: 0.7412
Epoch 167/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5346 - accuracy: 0.7404
Epoch 168/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5338 - a

858/858 [==============================] - 3s 4ms/step - loss: 0.5321 - accuracy: 0.7411
Epoch 239/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5322 - accuracy: 0.7415
Epoch 240/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5329 - accuracy: 0.7412
Epoch 241/400
858/858 [==============================] - 5s 6ms/step - loss: 0.5326 - accuracy: 0.7411
Epoch 242/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5327 - accuracy: 0.7413
Epoch 243/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5323 - accuracy: 0.7409
Epoch 244/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5327 - accuracy: 0.7410
Epoch 245/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5329 - accuracy: 0.7413
Epoch 246/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5326 - accuracy: 0.7412
Epoch 247/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5323 - a

858/858 [==============================] - 3s 4ms/step - loss: 0.5316 - accuracy: 0.7414
Epoch 318/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5314 - accuracy: 0.7416
Epoch 319/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5321 - accuracy: 0.7419
Epoch 320/400
858/858 [==============================] - 4s 5ms/step - loss: 0.5316 - accuracy: 0.7416
Epoch 321/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5316 - accuracy: 0.7419
Epoch 322/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5317 - accuracy: 0.7418
Epoch 323/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5315 - accuracy: 0.7417
Epoch 324/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5328 - accuracy: 0.7417
Epoch 325/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5353 - accuracy: 0.7414
Epoch 326/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5314 - a

858/858 [==============================] - 3s 4ms/step - loss: 0.5315 - accuracy: 0.7416
Epoch 397/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5321 - accuracy: 0.7422
Epoch 398/400
858/858 [==============================] - 3s 4ms/step - loss: 0.5335 - accuracy: 0.7411
Epoch 399/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5322 - accuracy: 0.7424
Epoch 400/400
858/858 [==============================] - 4s 4ms/step - loss: 0.5312 - accuracy: 0.7419
215/215 - 0s - loss: 0.5941 - accuracy: 0.7287
Loss:0.5941303968429565, Accuracy: 0.7287172079086304


In [34]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_optimzation.h5")